# Week1

## Excercise 1.1

In [102]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from typing import List
from enum import Enum
import unittest

class Turn(Enum):
    LEFT = 1
    RIGHT = 2
    ON_SEGMENT = 3
    ON_LINE = 4  # Colinear with the segment but not on the segment

class Intersection(Enum):
    INTERSECTS = 1
    NO_INTERSECTS = 2
    ON_BORDER = 3

class PolygonContainment(Enum):
    INSIDE = 1
    OUTSIDE = 2
    ON_BORDER = 3
    
class Point:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y

    x = 0
    y = 0
    
    def ToString(self):
        return str(self.x) + ", " + str(self.y)
    def __repr__(self):
        return self.ToString()
    def __str__(self):
        return self.ToString()
    
class Line:
    def __init__(self, p1 = Point(), p2 = Point()):
        self.p1 = p1
        self.p2 = p2
    
    p1 = Point()
    p2 = Point()
    
    def ToString(self):
        return "(" + self.p1.ToString() + "), (" + self.p2.ToString() + ")"       
    def __repr__(self):
        return self.ToString()
    def __str__(self):
        return self.ToString()

class Triangle:
    def __init__(self, a = Point(), b = Point(), c = Point()):
        self.a = a
        self.b = b
        self.c = c
    
    a = Point()
    b = Point()
    c = Point()
    
    def ToString(self):
        return "(" + self.a.ToString() + "), (" + self.b.ToString() + "), (" + self.c.ToString() + ")"       
    def __repr__(self):
        return self.ToString()
    def __str__(self):
        return self.ToString()
    
class Polygon:
    def __init__(self, verticies = List[Point]):
        self.verticies = verticies
    
    # Counterclockwise verticies of the polygon
    verticies = List[Point]
    
    def ToString(self):
        out = ""
        for p in verticies:
            out += "(" + p.ToString() + "), "
        return out
    
    def __repr__(self):
        return self.ToString()
    def __str__(self):
        return self.ToString()


In [103]:
import numpy as np

def between(e1, e2, test):
    return min(e1, e2) <= test and test <= max(e1, e2)

def FindTurn(line : Line, point : Point) -> Turn:
    m = np.array([[line.p1.x, line.p1.y, 1], [line.p2.x, line.p2.y, 1], [point.x, point.y, 1]])
    det = np.linalg.det(m)
    if abs(det - 0) < 1e-10:
        if between(line.p1.x, line.p2.x, point.x) and between(line.p1.y, line.p2.y, point.y):
            return Turn.ON_SEGMENT
        return Turn.ON_LINE
    elif det > 0:
        return Turn.LEFT
    return Turn.RIGHT

In [104]:
class TestNotebook(unittest.TestCase):

    def test_find_turn_example_1(self):
        l = Line(Point(-4, 4), Point(2, -2))
        p = Point(0, 0)
        self.assertEqual(FindTurn(l, p), Turn.ON_SEGMENT)
        
        p = Point(5, -5)
        self.assertEqual(FindTurn(l, p), Turn.ON_LINE)
        
        p = Point(1, 1)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(-1, -1)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
    
    def test_find_turn_example_2(self):
        l = Line(Point(-4, 0), Point(2, -10))
        
        p = Point(-8, 4)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
        p = Point(2, -7)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(9, 3)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(-4, 6)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(-8, -5)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
        p = Point(7, 1)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(2, 4)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        p = Point(-3, -9)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
    def test_find_turn_additional(self):
        # Vertical line
        l = Line(Point(0,0), Point(0,6))
        p = Point(0,3)
        self.assertEqual(FindTurn(l, p), Turn.ON_SEGMENT)
        
        p = Point(3,3)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
        p = Point(-1,3)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        l = Line(Point(0,6), Point(0,0))
        p = Point(3,3)
        self.assertEqual(FindTurn(l, p), Turn.LEFT)
        
        
        l = Line(Point(-3, -1), Point(-3, 4))
        p = Point(0, 0)
        self.assertEqual(FindTurn(l, p), Turn.RIGHT)
        
        


unittest.main(argv=[''], verbosity=2, exit=False)

test_find_turn_additional (__main__.TestNotebook) ... ok
test_find_turn_example_1 (__main__.TestNotebook) ... ok
test_find_turn_example_2 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


## Excercise 1.2

In [105]:
def TriangleViaTurn(t : Triangle, p : Point) -> PolygonContainment:
    turns = []
    # Following the triangle in 1 direction should result in all left or all right turns
    # (depending on the triange) if the point is inside.
    turns.append(FindTurn(Line(t.a, t.b), p))
    turns.append(FindTurn(Line(t.b, t.c), p))
    turns.append(FindTurn(Line(t.c, t.a), p))
    if Turn.ON_SEGMENT in turns:
        return PolygonContainment.ON_BORDER
    if all(elem == Turn.LEFT for elem in turns) or all(elem == Turn.RIGHT for elem in turns):
        return PolygonContainment.INSIDE
    return PolygonContainment.OUTSIDE

# https://www.youtube.com/watch?v=HYAgJN3x4GA
def TriangleViaVectors(t : Triangle, p : Point) -> PolygonContainment:
    w1 = (t.a.x * (t.c.y - t.a.y) + (p.y - t.a.y) * (t.c.x-t.a.x) - p.x * (t.c.y - t.a.y)) / (
        (t.b.y-t.a.y) * (t.c.x - t.a.x) - (t.b.x-t.a.x) * (t.c.y - t.a.y))
    w2 = (p.y - t.a.y - w1 * (t.b.y - t.a.y)) / (t.c.y-t.a.y)
    
    if (abs(w1 - 0) < 1e-10 or abs(w2 - 0) < 1e-10) and w1 <= 1 and w2 <= 1:
        return PolygonContainment.ON_BORDER
    # On side opposite A
    if w1 > 0 and w2 > 0 and abs(w1 + w2 - 1) < 1e-10:
        return PolygonContainment.ON_BORDER
    if w1 > 0 and w2 > 0 and w1 + w2 < 1:
        return PolygonContainment.INSIDE
    return PolygonContainment.OUTSIDE

def InsideTriangle(triangle : Triangle, point : Point) -> PolygonContainment:
#     return TriangleViaTurn(triangle, point)
    return TriangleViaVectors(triangle, point)

In [106]:
class TestNotebook(unittest.TestCase):

    def test_inside_triangle_1(self):
        t = Triangle(Point(0, 0), Point(5, 5), Point(0, 6))
        t2 = Triangle(Point(0, 0), Point(0, 6), Point(5, 5)) # Reverse area direction
        
        p = Point(3, 3)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
        
        p = Point(6, 6)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(2, 3)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
    
    def test_inside_triangle_2(self):
        t = Triangle(Point(1, 0), Point(0, 3), Point(3, 3))
        t2 = Triangle(Point(1, 0), Point(3, 3), Point(0, 3)) # Reverse area direction
        
        p = Point(0, 0)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(0, 1)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(0, 2)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(0, 3)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
        
        p = Point(1, 0)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
        
        p = Point(1, 1)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
        
        p = Point(1, 2)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
        
        p = Point(1, 3)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)
        
        p = Point(2, 0)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(2, 1)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.OUTSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.OUTSIDE)
        
        p = Point(2, 2)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.INSIDE)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.INSIDE)
        
        p = Point(2, 3)
        self.assertEqual(InsideTriangle(t, p), PolygonContainment.ON_BORDER)
        self.assertEqual(InsideTriangle(t2, p), PolygonContainment.ON_BORDER)

unittest.main(argv=[''], verbosity=2, exit=False)

test_inside_triangle_1 (__main__.TestNotebook) ... ok
test_inside_triangle_2 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


## Exercise 1.3 and 1.4

In [107]:
# Returns true if the ray eminating to the left (negative x) of p inserects the line segment l.
def RayIntersects(l : Line, p : Point) -> Intersection:
    # The ray insersects if the ray is between the segment's endpoints in y
    if not between(l.p1.y, l.p2.y, p.y):
        return False 
    # AND p is to the right of l
    # We need p1 below p2 to test for right turns
    l2 = l
    if l.p1.y > l.p2.y:
        l2 = Line(l.p2, l.p1)
    turn = FindTurn(l2, p)
    if turn == Turn.RIGHT:
        return Intersection.INTERSECTS
    elif turn == Turn.ON_SEGMENT:
        return Intersection.ON_BORDER
    return Intersection.NO_INTERSECTS
    
def InsidePolygon(poly : Polygon, q : Point) -> PolygonContainment :
    intersections = 0
    
    for i in range(len(poly.verticies)):
        l = Line(poly.verticies[i - 1], poly.verticies[i])
        intersection = RayIntersects(l, q)
        if intersection == Intersection.INTERSECTS:
            intersections += 1
        elif intersection == Intersection.ON_BORDER:
            return PolygonContainment.ON_BORDER
    
    return PolygonContainment.INSIDE if (intersections % 2 == 1) else PolygonContainment.OUTSIDE

In [109]:
class TestNotebook(unittest.TestCase):

    def test_inside_polygon_1(self):
        poly = Polygon([Point(-3, -1), Point(3, -1), Point(3, 5), Point(0, 2), Point(-3, 4)])
        
        q = Point(1, 3)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.ON_BORDER)
        
        q = Point(-1, 3)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.OUTSIDE)
        
        q = Point(0, 0)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
        
        q = Point(-2, 3)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
        
    def test_exercise_1_4(self):
        poly = Polygon([Point(-2, -3), Point(1, -4), Point(3, -2), Point(2, 1), Point(-2, 1)])
        
        q = Point(2, -3)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.ON_BORDER)
        
        q = Point(3, 0)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.OUTSIDE)
        
        q = Point(0, 0)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
        
        q = Point(2, -1)
        self.assertEqual(InsidePolygon(poly, q), PolygonContainment.INSIDE)
        

unittest.main(argv=[''], verbosity=2, exit=False)

test_exercise_1_4 (__main__.TestNotebook) ... ok
test_inside_polygon_1 (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK
